# 🏠 Real Estate Chatbot - Google Colab Version

This notebook runs the Streamlit app in Google Colab using ngrok to expose the web interface.


## Step 1: Install Dependencies


In [ ]:
!pip install -q streamlit>=1.28.0
!pip install -q llama-index>=0.10.0
!pip install -q llama-index-llms-openai>=0.1.0
!pip install -q llama-index-embeddings-openai>=0.1.0
!pip install -q openai>=1.0.0
!pip install -q pandas>=2.0.0
!pip install -q python-dotenv>=1.0.0
!pip install -q pypdf>=3.0.0
!pip install -q pyngrok


## Step 2: Set Up Project Structure


In [ ]:
import os
from pathlib import Path

# Create directory structure
os.makedirs("data/pdfs", exist_ok=True)

print("✅ Directory structure created")


## Step 3: Create Sample plots.csv


In [ ]:
import pandas as pd

# Create sample plots data
plots_data = {
    'Plot_ID': ['PLT-001', 'PLT-002', 'PLT-003', 'PLT-004', 'PLT-005'],
    'Area_SqFt': [2500, 3000, 2000, 3500, 2800],
    'Price': [1250000, 1500000, 1000000, 1750000, 1400000],
    'Status': ['Available', 'Sold', 'Available', 'Available', 'Reserved'],
    'Facing': ['North', 'East', 'South', 'West', 'North']
}

df = pd.DataFrame(plots_data)
df.to_csv('data/plots.csv', index=False)

print("✅ plots.csv created")
print(df)


## Step 4: Set Your OpenAI API Key

**Important:** Replace `your_openai_api_key_here` with your actual OpenAI API key


In [ ]:
# Set your OpenAI API key here
OPENAI_API_KEY = "your_openai_api_key_here"  # ⚠️ Replace with your actual key

# Create .env file
with open('.env', 'w') as f:
    f.write(f'OPENAI_API_KEY={OPENAI_API_KEY}')

print("✅ .env file created")
print("⚠️ Make sure to set your actual OpenAI API key above!")


## Step 5: Upload app.py

Upload your `app.py` file using the file uploader below, or copy-paste the code into a new cell.


In [ ]:
from google.colab import files

# Option 1: Upload app.py file
uploaded = files.upload()

# If app.py is uploaded, it will be in the current directory
if 'app.py' in uploaded:
    print("✅ app.py uploaded successfully")
else:
    print("⚠️ Please upload app.py file")


## Step 6: (Optional) Upload PDF Brochures

Upload PDF files to the `data/pdfs/` folder for document-based queries.


In [ ]:
from google.colab import files
import shutil

# Upload PDF files
uploaded_pdfs = files.upload()

# Move PDFs to data/pdfs folder
for filename in uploaded_pdfs.keys():
    if filename.endswith('.pdf'):
        shutil.move(filename, f'data/pdfs/{filename}')
        print(f"✅ Moved {filename} to data/pdfs/")

if not uploaded_pdfs:
    print("ℹ️ No PDFs uploaded. You can still query plot data.")


## Step 7: Run Streamlit with ngrok

This will start the Streamlit app and create a public URL using ngrok.


In [ ]:
from pyngrok import ngrok
import subprocess
import time

# Kill any existing Streamlit processes
try:
    subprocess.run(['pkill', '-f', 'streamlit'], check=False)
    time.sleep(2)
except:
    pass

# Start ngrok tunnel
port = 8501
public_url = ngrok.connect(port)
print(f"🌐 Public URL: {public_url}")
print(f"\n✅ Click the URL above to access your chatbot!")
print(f"\n⚠️ Note: The URL will change each time you restart ngrok.")
print(f"⚠️ Keep this cell running to keep the app accessible.")

# Run Streamlit in the background
!streamlit run app.py --server.port=8501 --server.address=0.0.0.0 --server.headless=true > /dev/null 2>&1 &

print("\n🚀 Streamlit app is starting...")
print("⏳ Please wait 10-15 seconds for the app to initialize.")
print(f"✅ Your app will be accessible at: {public_url}")


In [ ]:
# Check if Streamlit is running
import subprocess
result = subprocess.run(['pgrep', '-f', 'streamlit'], capture_output=True, text=True)
if result.stdout:
    print("✅ Streamlit is running!")
    print(f"🌐 Access your app at: {public_url}")
else:
    print("⚠️ Streamlit may not be running. Try running the previous cell again.")


## Step 8: Stop the App (when done)

Run this cell to stop the Streamlit app and close the ngrok tunnel.


In [ ]:
# Stop Streamlit
subprocess.run(['pkill', '-f', 'streamlit'], check=False)

# Disconnect ngrok
ngrok.kill()

print("✅ Streamlit app stopped and ngrok tunnel closed")
